<a href="https://colab.research.google.com/github/SokKanaTorajd/nlp-practices/blob/main/Tensor_with_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Setting Tensor Environment**

In [ ]:
# !pip uninstall tensorflow
# !pip install tensorflow-gpu==2.0.0-alpha0

In [ ]:
import tensorflow as tf

In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import operator
from collections import Counter
from wordcloud import WordCloud, STOPWORDS

from tqdm import tqdm
tqdm.pandas()

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.compat.v1.keras.layers import CuDNNLSTM
from tensorflow.keras import layers

%matplotlib inline
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)

In [ ]:
print(tf.__version__)

2.6.0


In [ ]:
x = tf.random.uniform([3, 3])

print("Is there a GPU available: "),
print(tf.test.is_gpu_available())

print("Is the Tensor on GPU #0:  "),
print(x.device.endswith('GPU:0'))

print("Device name: {}".format((x.device)))

Is there a GPU available: 
True
Is the Tensor on GPU #0:  
True
Device name: /job:localhost/replica:0/task:0/device:GPU:0


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/") 

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
cd gdrive/MyDrive/Colab\ Notebooks/data

[Errno 2] No such file or directory: 'gdrive/MyDrive/Colab Notebooks/data'
/content/gdrive/MyDrive/Colab Notebooks/data


In [ ]:
!pip install gdown

In [ ]:
ls -l

total 829491
-rw------- 1 root root     35389 Oct  1 15:18  dataset_tweet_sentimen_tayangan_tv.csv
-rw------- 1 root root 102699149 Oct  1 07:51  glove_wiki_id_50.txt
-rw------- 1 root root    961247 Oct  1 07:31 'Indonesia Sentiment.csv'
-rw------- 1 root root 745700928 Oct  1 16:00  w2vec_wiki_id_case.wv.vectors.npy


In [ ]:
import pandas as pd
df = pd.read_csv("Indonesia Sentiment.csv")
df = df.drop_duplicates()

In [ ]:
X_train, X_test  = train_test_split(df, test_size=0.1, random_state=42)

In [ ]:
y_train, y_test = X_train['sentimen'].values, X_test['sentimen'].values

In [ ]:
embed_size = 300 # how big is each word vector
max_features = 50000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a question to use

In [ ]:
X_train_data = X_train['Tweet'].fillna('_NA_').values
Y_train_data = y_train
X_test_data = X_test['Tweet'].fillna('_NA_').values
Y_test_data = y_test
# X_submission = df_test['Tweet'].fillna('_NA_').values

In [ ]:
X_train_data.shape

(9231,)

In [ ]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train_data))
X_train = tokenizer.texts_to_sequences(X_train_data)
X_test = tokenizer.texts_to_sequences(X_test_data)
# X_submission = tokenizer.texts_to_sequences(X_submission)

In [ ]:
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)
# X_submission = pad_sequences(X_submission, maxlen=maxlen)

In [ ]:
model1 = Sequential()
model1.add(Embedding(max_features, embed_size, input_length=maxlen))
model1.add(Bidirectional(CuDNNLSTM(128, return_sequences=True)))
model1.add(GlobalMaxPool1D())
model1.add(Dropout(0.2))
model1.add(Dense(64, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(32, activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model1.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 300)          15000000  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 100, 256)          440320    
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 256)               0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 64)                16448     
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 32)               

In [ ]:
%time model1.fit(X_train, y_train, batch_size=512, epochs=10, validation_data=(X_test, y_test), verbose = 1)

Epoch 1/10
19/19 [==============================] - 6s 179ms/step - loss: 0.1061 - accuracy: 0.4918 - val_loss: 0.1468 - val_accuracy: 0.4815
Epoch 2/10
19/19 [==============================] - 3s 146ms/step - loss: -0.3098 - accuracy: 0.4972 - val_loss: 0.0943 - val_accuracy: 0.4815
Epoch 3/10
19/19 [==============================] - 3s 146ms/step - loss: -2.3608 - accuracy: 0.4972 - val_loss: -1.6086 - val_accuracy: 0.4815
Epoch 4/10
19/19 [==============================] - 3s 145ms/step - loss: -12.1304 - accuracy: 0.4972 - val_loss: -3.0388 - val_accuracy: 0.4815
Epoch 5/10
19/19 [==============================] - 3s 146ms/step - loss: -37.0389 - accuracy: 0.5083 - val_loss: -16.2452 - val_accuracy: 0.4815
Epoch 6/10
19/19 [==============================] - 3s 146ms/step - loss: -94.6063 - accuracy: 0.5072 - val_loss: -38.1695 - val_accuracy: 0.4815
Epoch 7/10
19/19 [==============================] - 3s 146ms/step - loss: -188.0441 - accuracy: 0.5341 - val_loss: -54.8231 - val_accu

# Glove Load

In [ ]:
print(len(X_train_data))

9231


In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

all_words = []
for sent in X_train_data:
    tokenize_word = word_tokenize(sent)
    for word in tokenize_word:
        all_words.append(word)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
unique_words = set(all_words)
print(len(unique_words))

21625


In [ ]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

In [ ]:
vocab_length = 21750
embedded_sentences = [one_hot(sent, vocab_length) for sent in X_train_data]
print(embedded_sentences )

[[5666, 21270, 12462, 5482, 4133, 8835, 1652, 14336, 1662, 6140, 11789, 16758, 1662, 18807, 6607, 6607], [7671, 9441, 21708, 15033, 20975, 15707, 16170, 14536, 8131, 8188, 13338, 8406, 18692, 14331], [3325, 12631, 17071, 5533, 17071, 2078, 9600, 10624, 3531, 11990, 16092, 14544, 8986, 66, 4180, 1299, 18652, 13338, 5279, 878, 5482, 16059], [5026, 16349, 2999, 21097, 11372, 20173, 11639, 5390, 5026, 12491, 13671, 6501, 5026, 13166, 4463], [6498, 6890, 6879, 100, 20857, 13648, 540, 15765, 11578, 4405, 9951], [13737, 15958, 8956, 12624, 21357, 6890, 10739, 12479, 21386, 1048, 6176, 3846, 16747, 12479, 14233, 21214, 6879, 21699], [9059, 12880, 9059, 1406, 16316, 17305, 2712, 2852, 44, 3052, 6498, 9072, 8358, 19806], [20140, 4788, 15794, 19190, 19190, 3474, 14139, 10045, 18099, 7425, 14171, 1818, 11669, 10953, 8358, 16805, 6603, 20302, 18509, 12022, 1624], [12363, 13664, 18509, 4411, 13307, 881, 11841, 15784, 6498, 13664, 220], [13788, 13338, 11028, 8834, 12021, 7716, 12716, 4153, 11514, 201

In [ ]:
word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(X_train_data, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

In [ ]:
padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')
print(padded_sentences)

[[ 5666 21270 12462 ...     0     0     0]
 [ 7671  9441 21708 ...     0     0     0]
 [ 3325 12631 17071 ...     0     0     0]
 ...
 [18543 12321 14139 ...     0     0     0]
 [15765 13797  6501 ...     0     0     0]
 [16805 19338 10395 ...     0     0     0]]


In [ ]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(X_train_data)

In [ ]:
vocab_length = len(word_tokenizer.word_index) + 1

In [ ]:
embedded_sentences = word_tokenizer.texts_to_sequences(X_train_data)
print(embedded_sentences)

[[725, 190, 5352, 113, 437, 909, 8580, 1786, 45, 3894, 1995, 438, 45, 3147, 1068, 1068], [8581, 461, 3148, 8582, 8583, 439, 8584, 3895, 8585, 8586, 3, 3896, 910, 248], [8587, 138, 7, 8588, 7, 8589, 598, 1394, 5353, 755, 1395, 5354, 9, 8590, 8591, 234, 512, 3, 167, 8592, 113, 209], [4, 8593, 408, 5355, 1787, 2629, 1788, 599, 4, 53, 280, 26, 4, 8594, 8595], [1, 10, 36, 1212, 287, 103, 8596, 46, 1069, 263, 8597], [84, 1134, 6, 49, 536, 10, 100, 5356, 911, 5357, 1070, 25, 1789, 5356, 814, 8598, 36, 2630], [14, 61, 14, 3897, 31, 8599, 1011, 291, 8600, 862, 1, 3898, 27, 18], [8, 13, 112, 815, 815, 19, 310, 1213, 52, 173, 513, 863, 54, 2261, 27, 5, 17, 68, 41, 150, 8601], [783, 51, 41, 816, 2262, 8602, 12, 477, 1, 51, 5358], [5359, 3, 696, 62, 498, 2263, 8603, 1071, 135, 8, 462, 8604, 8605, 478, 598, 16, 32, 135], [80, 2, 1135, 3149, 1072, 2631], [5360, 20, 145, 23, 1136, 1639, 1790, 1639, 5361, 2632, 37, 571, 159, 6, 130, 3899, 420, 81, 1996, 962, 349, 614, 11], [43, 201, 333, 756, 1997, 860

In [ ]:
from nltk.tokenize import word_tokenize

word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(X_train_data, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')

print(padded_sentences)

[[  725   190  5352 ...     0     0     0]
 [ 8581   461  3148 ...     0     0     0]
 [ 8587   138     7 ...     0     0     0]
 ...
 [  154   471   310 ...     0     0     0]
 [   46   685    26 ...     0     0     0]
 [    5   404 21234 ...     0     0     0]]


In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('glove_wiki_id_50.txt', encoding="utf8")

In [ ]:
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions

glove_file.close()

In [ ]:
embedding_matrix = zeros((vocab_length, 50))
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
embed_size = 50 # how big is each word vector
max_features = 21236 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a tweet to use

model2 = Sequential()
model2.add(Embedding(max_features, embed_size, input_length=maxlen, weights = [embedding_matrix]))
model2.add(Bidirectional(CuDNNLSTM(128, return_sequences=True)))
model2.add(GlobalMaxPool1D())
model2.add(Dropout(0.2))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(32, activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model2.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 50)           1061800   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 100, 256)          184320    
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 256)               0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 64)                16448     
_________________________________________________________________
dropout_10 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 32)               

In [ ]:
%time model2.fit(X_train, y_train, batch_size=512, epochs=50, validation_data=(X_test, y_test), verbose = 1)

Epoch 1/50
19/19 [==============================] - 5s 140ms/step - loss: 0.3055 - accuracy: 0.4851 - val_loss: 0.1378 - val_accuracy: 0.4815
Epoch 2/50
19/19 [==============================] - 2s 108ms/step - loss: -0.3705 - accuracy: 0.4972 - val_loss: 0.3665 - val_accuracy: 0.4815
Epoch 3/50
19/19 [==============================] - 2s 110ms/step - loss: -0.8515 - accuracy: 0.4972 - val_loss: 0.6121 - val_accuracy: 0.4815
Epoch 4/50
19/19 [==============================] - 2s 107ms/step - loss: -1.1764 - accuracy: 0.4972 - val_loss: 0.8117 - val_accuracy: 0.4815
Epoch 5/50
19/19 [==============================] - 2s 107ms/step - loss: -1.6541 - accuracy: 0.4972 - val_loss: 0.9039 - val_accuracy: 0.4815
Epoch 6/50
19/19 [==============================] - 2s 111ms/step - loss: -3.0467 - accuracy: 0.4972 - val_loss: 0.5815 - val_accuracy: 0.4815
Epoch 7/50
19/19 [==============================] - 2s 108ms/step - loss: -19.0483 - accuracy: 0.4972 - val_loss: -7.9847 - val_accuracy: 0.481